# Day 4: Advanced RAG with Multiple Sources and Citations

**Learning Goals:**
- Query multiple document sources simultaneously
- Implement source attribution and citations in responses
- Use metadata filtering for targeted retrieval
- Build a multi-document RAG system with proper citations
- Learn reranking techniques for better retrieval
- Understand retrieval quality metrics

**Time:** 2-3 hours

---

## What We'll Build Today

1. **Multi-Source RAG**: Query different document collections
2. **Source Attribution**: Track which documents were used in answers
3. **Metadata Filtering**: Search specific document types or authors
4. **Citation System**: Add proper citations to LLM responses
5. **Reranking**: Improve retrieval accuracy with reranking
6. **Retrieval Evaluation**: Measure retrieval quality

## Part 1: Environment Setup

Load API keys and initialize components (same as previous days).

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
LANGCHAIN_API_KEY  = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING  = os.getenv("LANGCHAIN_TRACING_V2", "false") == "true"

print("✅ OpenRouter key loaded" if OPENROUTER_API_KEY else "⚠️  Missing OPENROUTER_API_KEY")
print("✅ LangSmith tracing enabled" if LANGCHAIN_TRACING else "ℹ️  LangSmith tracing disabled")

✅ OpenRouter key loaded
✅ LangSmith tracing enabled


In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(
    model="openai/gpt-3.5-turbo",
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0.7,
)

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",
)

print("✅ LLM and Embeddings ready")

✅ LLM and Embeddings ready


## Part 2: Building a Multi-Source Document Collection

**Why Multiple Sources?**
- Different document types (research papers, blog posts, documentation)
- Different authors or perspectives
- Time-based filtering (recent vs historical)
- Domain-specific collections

Let's create a diverse document collection with rich metadata.

### 2a. Creating Documents with Rich Metadata

Metadata is crucial for advanced RAG - it enables filtering, source attribution, and citations.

In [3]:
from langchain_core.documents import Document
from datetime import datetime

# Create documents from different sources with rich metadata
documents = [
    # Research Papers on AI
    Document(
        page_content="Large Language Models (LLMs) like GPT-4 demonstrate emergent abilities in reasoning, comprehension, and generation. These models are trained on diverse internet text and can perform tasks they weren't explicitly trained for.",
        metadata={
            "source": "research_paper",
            "title": "Emergent Abilities of Large Language Models",
            "author": "Dr. Jane Smith",
            "year": 2023,
            "category": "artificial_intelligence",
            "document_type": "paper",
            "page": 1
        }
    ),
    Document(
        page_content="Retrieval Augmented Generation (RAG) combines the power of large language models with external knowledge retrieval. This approach significantly reduces hallucinations and provides more accurate, up-to-date information.",
        metadata={
            "source": "research_paper",
            "title": "RAG: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks",
            "author": "Dr. John Doe",
            "year": 2023,
            "category": "artificial_intelligence",
            "document_type": "paper",
            "page": 3
        }
    ),
    Document(
        page_content="Vector databases enable efficient similarity search over high-dimensional embeddings. Popular options include Pinecone, Weaviate, Qdrant, and ChromaDB, each offering different trade-offs in performance and features.",
        metadata={
            "source": "research_paper",
            "title": "Vector Databases for AI Applications",
            "author": "Dr. Jane Smith",
            "year": 2024,
            "category": "databases",
            "document_type": "paper",
            "page": 2
        }
    ),
    
    # Technical Blog Posts
    Document(
        page_content="LangChain provides a comprehensive framework for building LLM applications. It includes modules for prompts, chains, agents, memory, and retrieval, making it easier to develop complex AI systems.",
        metadata={
            "source": "blog_post",
            "title": "Getting Started with LangChain",
            "author": "Tech Blogger",
            "year": 2024,
            "category": "software_engineering",
            "document_type": "blog",
            "url": "https://techblog.example/langchain-intro"
        }
    ),
    Document(
        page_content="When building production RAG systems, consider chunking strategies, embedding models, retrieval algorithms, and prompt engineering. Each choice significantly impacts the quality of your results.",
        metadata={
            "source": "blog_post",
            "title": "Production RAG Best Practices",
            "author": "ML Engineer",
            "year": 2024,
            "category": "software_engineering",
            "document_type": "blog",
            "url": "https://mlblog.example/rag-best-practices"
        }
    ),
    
    # Documentation
    Document(
        page_content="ChromaDB is an open-source embedding database. It's designed to be simple, fast, and easy to use. Install with: pip install chromadb. Basic usage involves creating a client, creating a collection, and adding documents.",
        metadata={
            "source": "documentation",
            "title": "ChromaDB Documentation - Quick Start",
            "author": "ChromaDB Team",
            "year": 2024,
            "category": "databases",
            "document_type": "documentation",
            "version": "0.4.0"
        }
    ),
    Document(
        page_content="LangChain's RetrievalQA chain combines a retriever with a question-answering chain. It retrieves relevant documents and passes them to an LLM to generate answers based on the retrieved context.",
        metadata={
            "source": "documentation",
            "title": "LangChain Documentation - RetrievalQA",
            "author": "LangChain Team",
            "year": 2024,
            "category": "software_engineering",
            "document_type": "documentation",
            "version": "0.1.0"
        }
    ),
    
    # News Articles
    Document(
        page_content="OpenAI announced GPT-4 Turbo with improved performance and reduced costs. The new model features a 128K context window and knowledge cutoff of April 2023, making it more capable for complex tasks.",
        metadata={
            "source": "news_article",
            "title": "OpenAI Releases GPT-4 Turbo",
            "author": "Tech News Reporter",
            "year": 2024,
            "category": "artificial_intelligence",
            "document_type": "news",
            "publication": "Tech Daily"
        }
    ),
]

print(f"✅ Created {len(documents)} documents from {len(set(d.metadata['document_type'] for d in documents))} different source types")
print(f"📚 Document types: {', '.join(set(d.metadata['document_type'] for d in documents))}")

✅ Created 8 documents from 4 different source types
📚 Document types: news, documentation, blog, paper


### 2b. Storing Documents in ChromaDB with Metadata

Let's store these documents and preserve all metadata for later filtering and citation.

In [4]:
from langchain_community.vectorstores import Chroma

# Create a new ChromaDB collection for multi-source documents
vectorstore = Chroma(
    collection_name="multi_source_docs",
    embedding_function=embeddings,
    persist_directory="./chroma_db_day4_sources"
)

# Add documents with metadata
vectorstore.add_documents(documents)

print(f"✅ Stored {len(documents)} documents in ChromaDB")
print(f"📊 Collection name: multi_source_docs")

/var/folders/2_/vpdn53cj1bl7w9l8mgd5wnxr9bfspm/T/ipykernel_20976/1584831788.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


✅ Stored 8 documents in ChromaDB
📊 Collection name: multi_source_docs


## Part 3: Metadata Filtering for Targeted Retrieval

**Why Filter?**
- Query specific document types (only papers, only docs)
- Search by author, year, or category
- Improve relevance by narrowing scope
- Create specialized retrieval pipelines

### 3a. Filtering by Document Type

Let's retrieve only from research papers.

In [5]:
# Search only research papers
query = "What are the benefits of using large language models?"

# Apply metadata filter
results = vectorstore.similarity_search(
    query,
    k=3,
    filter={"document_type": "paper"}
)

print("🔍 Search Results (Papers Only):\n")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.metadata['title']}")
    print(f"   Author: {doc.metadata['author']}")
    print(f"   Type: {doc.metadata['document_type']}")
    print(f"   Content: {doc.page_content[:100]}...")
    print()

🔍 Search Results (Papers Only):

1. Emergent Abilities of Large Language Models
   Author: Dr. Jane Smith
   Type: paper
   Content: Large Language Models (LLMs) like GPT-4 demonstrate emergent abilities in reasoning, comprehension, ...

2. RAG: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
   Author: Dr. John Doe
   Type: paper
   Content: Retrieval Augmented Generation (RAG) combines the power of large language models with external knowl...

3. Vector Databases for AI Applications
   Author: Dr. Jane Smith
   Type: paper
   Content: Vector databases enable efficient similarity search over high-dimensional embeddings. Popular option...



### 3b. Filtering by Multiple Criteria

Combine multiple metadata filters for precise retrieval.

**Note**: For multiple conditions, use `$and` operator:
```python
filter={"$and": [{"document_type": "documentation"}, {"year": 2024}]}
```

For OR conditions, use `$or`:
```python
filter={"$or": [{"document_type": "paper"}, {"document_type": "blog"}]}
```

For IN conditions, use `$in`:
```python
filter={"document_type": {"$in": ["blog", "documentation"]}}
```

In [10]:
# Search only recent documentation (2024)
query = "How do I use ChromaDB?"

results = vectorstore.similarity_search(
    query,
    k=2,
    filter={
    "$and": [
        {"document_type": "documentation"},
        {"year": 2024}
    ]
}
)

print("🔍 Search Results (2024 Documentation Only):\n")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.metadata['title']}")
    print(f"   Type: {doc.metadata['document_type']} ({doc.metadata['year']})")
    print(f"   Content: {doc.page_content[:120]}...")
    print()

🔍 Search Results (2024 Documentation Only):

1. ChromaDB Documentation - Quick Start
   Type: documentation (2024)
   Content: ChromaDB is an open-source embedding database. It's designed to be simple, fast, and easy to use. Install with: pip inst...

2. LangChain Documentation - RetrievalQA
   Type: documentation (2024)
   Content: LangChain's RetrievalQA chain combines a retriever with a question-answering chain. It retrieves relevant documents and ...



### 3c. Filtering by Author

Find all documents from a specific author.

In [11]:
# Search documents by Dr. Jane Smith
query = "vector databases"

results = vectorstore.similarity_search(
    query,
    k=3,
    filter={"author": "Dr. Jane Smith"}
)

print("🔍 Search Results (Dr. Jane Smith's documents):\n")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.metadata['title']}")
    print(f"   Year: {doc.metadata['year']}")
    print(f"   Category: {doc.metadata['category']}")
    print(f"   Content: {doc.page_content[:100]}...")
    print()

🔍 Search Results (Dr. Jane Smith's documents):

1. Vector Databases for AI Applications
   Year: 2024
   Category: databases
   Content: Vector databases enable efficient similarity search over high-dimensional embeddings. Popular option...

2. Emergent Abilities of Large Language Models
   Year: 2023
   Category: artificial_intelligence
   Content: Large Language Models (LLMs) like GPT-4 demonstrate emergent abilities in reasoning, comprehension, ...



## Part 4: Building a RAG System with Citations

**Why Citations Matter?**
- Transparency and trust
- Fact verification
- Follow-up reading
- Academic/professional credibility

### 4a. Simple RAG with Source Attribution

First, let's build a basic RAG that includes sources.

In [12]:
from langchain_classic.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

# Create a custom prompt that encourages source mention
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {question}

Answer: Provide a comprehensive answer and mention which sources you used."""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 4}),
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# Query the system
question = "What is RAG and why is it useful?"
result = qa_chain.invoke({"query": question})

print("❓ Question:", question)
print("\n📝 Answer:", result['result'])
print("\n📚 Sources Used:")
for i, doc in enumerate(result['source_documents'], 1):
    print(f"\n{i}. {doc.metadata['title']}")
    print(f"   By {doc.metadata['author']} ({doc.metadata['year']})")
    print(f"   Type: {doc.metadata['document_type']}")

❓ Question: What is RAG and why is it useful?

📝 Answer: RAG, or Retrieval Augmented Generation, is an approach that combines the power of large language models with external knowledge retrieval. This method significantly reduces hallucinations and provides more accurate, up-to-date information by leveraging both the capabilities of language models and external sources of knowledge. This approach is useful because it helps improve the quality of generated content by incorporating relevant information from external sources, thus enhancing the accuracy and reliability of the generated output.

Sources used: The provided context snippets about RAG and LangChain.

📚 Sources Used:

1. RAG: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
   By Dr. John Doe (2023)
   Type: paper

2. Production RAG Best Practices
   By ML Engineer (2024)
   Type: blog

3. LangChain Documentation - RetrievalQA
   By LangChain Team (2024)
   Type: documentation

4. Getting Started with LangChain

### 4b. Advanced Citation System

Now let's build a proper citation system with numbered references.

In [14]:
from langchain_core.documents import Document
from typing import List

def format_citations(documents: List[Document]) -> tuple[str, List[dict]]:
    """
    Format documents with citation numbers and return formatted context + citation list
    """
    citations = []
    formatted_chunks = []
    
    for i, doc in enumerate(documents, 1):
        # Create citation entry
        citation = {
            "number": i,
            "title": doc.metadata.get("title", "Untitled"),
            "author": doc.metadata.get("author", "Unknown"),
            "year": doc.metadata.get("year", "N/A"),
            "type": doc.metadata.get("document_type", "unknown"),
        }
        
        # Add optional metadata
        if "page" in doc.metadata:
            citation["page"] = doc.metadata["page"]
        if "url" in doc.metadata:
            citation["url"] = doc.metadata["url"]
        
        citations.append(citation)
        
        # Format chunk with citation number
        formatted_chunks.append(f"[{i}] {doc.page_content}")
    
    context = "\n\n".join(formatted_chunks)
    return context, citations

def format_citation_list(citations: List[dict]) -> str:
    """Format citations as a readable reference list"""
    citation_text = "\n\nReferences:\n"
    for cite in citations:
        cite_str = f"[{cite['number']}] {cite['title']} - {cite['author']} ({cite['year']})"
        if 'page' in cite:
            cite_str += f", p. {cite['page']}"
        if 'url' in cite:
            cite_str += f"\n    {cite['url']}"
        citation_text += f"{cite_str}\n"
    return citation_text

# Test the citation system
question = "Explain how vector databases work in AI applications"

# Retrieve documents
retrieved_docs = vectorstore.similarity_search(question, k=4)

# Format with citations
context, citations = format_citations(retrieved_docs)

print("📄 Formatted Context with Citations:\n")
print(context[:500] + "...")
print("\n" + "="*70)
print(format_citation_list(citations))

📄 Formatted Context with Citations:

[1] Vector databases enable efficient similarity search over high-dimensional embeddings. Popular options include Pinecone, Weaviate, Qdrant, and ChromaDB, each offering different trade-offs in performance and features.

[2] Retrieval Augmented Generation (RAG) combines the power of large language models with external knowledge retrieval. This approach significantly reduces hallucinations and provides more accurate, up-to-date information.

[3] When building production RAG systems, consider chun...



References:
[1] Vector Databases for AI Applications - Dr. Jane Smith (2024), p. 2
[2] RAG: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks - Dr. John Doe (2023), p. 3
[3] Production RAG Best Practices - ML Engineer (2024)
    https://mlblog.example/rag-best-practices
[4] Getting Started with LangChain - Tech Blogger (2024)
    https://techblog.example/langchain-intro



### 4c. Complete RAG Chain with Citations

Integrate everything into a RAG chain that produces cited answers.

**Python Tip**: The `**search_kwargs` syntax unpacks a dictionary as keyword arguments. It's not pointers - it's a way to dynamically pass parameters to functions.

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

class CitedRAG:
    """RAG system that provides answers with proper citations"""
    
    def __init__(self, llm, vectorstore, k=4):
        self.llm = llm
        self.vectorstore = vectorstore
        self.k = k
        
        # Prompt that expects citations
        self.prompt = PromptTemplate(
            template="""You are a helpful assistant that provides accurate answers based on provided sources.
Use the context below to answer the question. Each piece of context is marked with a citation number [1], [2], etc.

When you use information from a source, cite it using the reference number in square brackets, like [1] or [2].
You can cite multiple sources like [1,2] if information comes from multiple sources.

If the context doesn't contain enough information to answer the question, say so clearly.

Context:
{context}

Question: {question}

Provide a clear, well-cited answer:""",
            input_variables=["context", "question"]
        )
        
        # Create LCEL chain (modern LangChain approach)
        self.chain = (
            {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
            | self.prompt
            | self.llm
            | StrOutputParser()
        )
    
    def query(self, question: str, filter_dict: dict = None):
        """Query the RAG system and return answer with citations"""
        
        # Retrieve relevant documents
        search_kwargs = {"k": self.k}
        if filter_dict:
            search_kwargs["filter"] = filter_dict
            
        # **search_kwargs unpacks the dict as keyword arguments
        # Equivalent to: similarity_search(question, k=4, filter=filter_dict)
        retrieved_docs = self.vectorstore.similarity_search(
            question, 
            **search_kwargs
        )
        
        # Format with citations
        context, citations = format_citations(retrieved_docs)
        
        # Generate answer
        answer = self.chain.invoke({"context": context, "question": question})
        
        # Format final output
        result = {
            "question": question,
            "answer": answer,
            "citations": citations,
            "source_documents": retrieved_docs
        }
        
        return result
    
    def print_result(self, result):
        """Pretty print the result"""
        print("❓ Question:", result['question'])
        print("\n📝 Answer:")
        print(result['answer'])
        print(format_citation_list(result['citations']))

# Create CitedRAG instance
cited_rag = CitedRAG(llm, vectorstore, k=4)

# Test it!
question = "What are the best practices for building production RAG systems?"
result = cited_rag.query(question)
cited_rag.print_result(result)

❓ Question: What are the best practices for building production RAG systems?

📝 Answer:
The best practices for building production RAG systems include considering chunking strategies, embedding models, retrieval algorithms, and prompt engineering. Each of these choices significantly impacts the quality of the results obtained from the system [1].


References:
[1] Production RAG Best Practices - ML Engineer (2024)
    https://mlblog.example/rag-best-practices
[2] RAG: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks - Dr. John Doe (2023), p. 3
[3] Getting Started with LangChain - Tech Blogger (2024)
    https://techblog.example/langchain-intro
[4] LangChain Documentation - RetrievalQA - LangChain Team (2024)



### 4d. Query with Filters and Citations

Combine metadata filtering with citations.

**Filter Syntax Examples:**
- Single filter: `{"document_type": "paper"}`
- Multiple AND: `{"$and": [{"document_type": "documentation"}, {"year": 2024}]}`
- OR condition: `{"$or": [{"author": "Dr. Jane Smith"}, {"author": "Dr. John Doe"}]}`
- IN condition: `{"document_type": {"$in": ["blog", "documentation"]}}`

In [16]:
# Query only from blog posts and documentation
question = "How do I get started with LangChain?"

result = cited_rag.query(
    question,
    filter_dict={"document_type": {"$in": ["blog", "documentation"]}}
)

cited_rag.print_result(result)

❓ Question: How do I get started with LangChain?

📝 Answer:
To get started with LangChain, you can begin by exploring its comprehensive framework designed for building LLM applications. This framework includes modules for prompts, chains, agents, memory, and retrieval, which collectively simplify the development of complex AI systems [1]. By leveraging LangChain's RetrievalQA chain, which combines a retriever with a question-answering chain, you can retrieve relevant documents and pass them to an LLM to generate answers based on the retrieved context [2]. This integration of modules within LangChain facilitates the creation of advanced AI systems.


References:
[1] Getting Started with LangChain - Tech Blogger (2024)
    https://techblog.example/langchain-intro
[2] LangChain Documentation - RetrievalQA - LangChain Team (2024)
[3] ChromaDB Documentation - Quick Start - ChromaDB Team (2024)
[4] Production RAG Best Practices - ML Engineer (2024)
    https://mlblog.example/rag-best-practic

## Part 5: Multi-Retriever RAG

**Advanced Pattern: Multiple Retrievers**
- Query multiple collections simultaneously
- Combine results from different sources
- Useful for diverse knowledge bases

### 5a. Creating Separate Collections

Let's simulate having separate vector stores for different document types.

In [17]:
# Create separate vector stores for different document types
papers_docs = [d for d in documents if d.metadata['document_type'] == 'paper']
blog_docs = [d for d in documents if d.metadata['document_type'] == 'blog']
doc_docs = [d for d in documents if d.metadata['document_type'] == 'documentation']

# Create separate vector stores
papers_store = Chroma(
    collection_name="papers",
    embedding_function=embeddings,
    persist_directory="./chroma_db_day4_papers"
)
papers_store.add_documents(papers_docs)

blog_store = Chroma(
    collection_name="blogs",
    embedding_function=embeddings,
    persist_directory="./chroma_db_day4_blogs"
)
blog_store.add_documents(blog_docs)

doc_store = Chroma(
    collection_name="documentation",
    embedding_function=embeddings,
    persist_directory="./chroma_db_day4_docs"
)
doc_store.add_documents(doc_docs)

print(f"✅ Created 3 separate collections:")
print(f"   📄 Papers: {len(papers_docs)} documents")
print(f"   📝 Blogs: {len(blog_docs)} documents")
print(f"   📚 Documentation: {len(doc_docs)} documents")

✅ Created 3 separate collections:
   📄 Papers: 3 documents
   📝 Blogs: 2 documents
   📚 Documentation: 2 documents


### 5b. Ensemble Retriever (Multi-Source)

Combine multiple retrievers to search across all sources.

In [32]:
from langchain_classic.retrievers import EnsembleRetriever

# Create individual retrievers
papers_retriever = papers_store.as_retriever(search_kwargs={"k": 2})
blog_retriever = blog_store.as_retriever(search_kwargs={"k": 2})
doc_retriever = doc_store.as_retriever(search_kwargs={"k": 2})

# Combine into ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[papers_retriever, blog_retriever, doc_retriever],
    weights=[0.4, 0.3, 0.3]  # Give slightly more weight to papers
)

# Test the ensemble retriever
# NOTE: Using invoke() instead of get_relevant_documents() (modern LangChain practice)
question = "What is LangChain and how does it help with RAG?"
results = ensemble_retriever.invoke(question)

print(f"🔍 Retrieved {len(results)} documents from multiple sources:\n")
for i, doc in enumerate(results, 1):
    print(f"{i}. [{doc.metadata['document_type'].upper()}] {doc.metadata['title']}")
    print(f"   Content: {doc.page_content[:100]}...")
    print()

🔍 Retrieved 6 documents from multiple sources:

1. [PAPER] RAG: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
   Content: Retrieval Augmented Generation (RAG) combines the power of large language models with external knowl...

2. [PAPER] Emergent Abilities of Large Language Models
   Content: Large Language Models (LLMs) like GPT-4 demonstrate emergent abilities in reasoning, comprehension, ...

3. [BLOG] Getting Started with LangChain
   Content: LangChain provides a comprehensive framework for building LLM applications. It includes modules for ...

4. [DOCUMENTATION] LangChain Documentation - RetrievalQA
   Content: LangChain's RetrievalQA chain combines a retriever with a question-answering chain. It retrieves rel...

5. [BLOG] Production RAG Best Practices
   Content: When building production RAG systems, consider chunking strategies, embedding models, retrieval algo...

6. [DOCUMENTATION] ChromaDB Documentation - Quick Start
   Content: ChromaDB is an open-sourc

### 5c. Multi-Source RAG with Citations

Combine ensemble retrieval with our citation system.

In [33]:
from langchain_classic.retrievers import EnsembleRetriever
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

class MultiSourceRAG:
    """RAG system that queries multiple sources and provides citations"""
    
    def __init__(self, llm, retrievers_dict, default_k=2):
        """
        Args:
            llm: Language model
            retrievers_dict: Dict of {source_name: (vectorstore, weight)}
            default_k: Number of docs to retrieve per source
        """
        self.llm = llm
        self.retrievers_dict = retrievers_dict
        self.default_k = default_k
        
        # Create ensemble retriever
        retrievers = []
        weights = []
        for source_name, (vectorstore, weight) in retrievers_dict.items():
            retrievers.append(vectorstore.as_retriever(search_kwargs={"k": default_k}))
            weights.append(weight)
        
        self.ensemble_retriever = EnsembleRetriever(
            retrievers=retrievers,
            weights=weights
        )
        
        self.prompt = PromptTemplate(
            template="""You are a research assistant that provides well-cited answers from multiple sources.

Use the context below to answer the question. Each piece of context has a citation number [1], [2], etc.
Always cite your sources using these numbers when making claims.

Context from multiple sources:
{context}

Question: {question}

Provide a comprehensive, well-cited answer:""",
            input_variables=["context", "question"]
        )
        
        # Create LCEL chain (modern LangChain approach)
        self.chain = (
            {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
            | self.prompt
            | self.llm
            | StrOutputParser()
        )
    
    def query(self, question: str):
        """Query across all sources"""
        
        # Retrieve from all sources
        retrieved_docs = self.ensemble_retriever.invoke(question)
        
        # Format with citations
        context, citations = format_citations(retrieved_docs)
        
        # Generate answer
        answer = self.chain.invoke({"context": context, "question": question})
        
        return {
            "question": question,
            "answer": answer,
            "citations": citations,
            "source_documents": retrieved_docs,
            "source_breakdown": self._get_source_breakdown(retrieved_docs)
        }
    
    def _get_source_breakdown(self, docs):
        """Count documents by source type"""
        breakdown = {}
        for doc in docs:
            doc_type = doc.metadata.get('document_type', 'unknown')
            breakdown[doc_type] = breakdown.get(doc_type, 0) + 1
        return breakdown
    
    def print_result(self, result):
        """Pretty print with source breakdown"""
        print("❓ Question:", result['question'])
        print("\n📝 Answer:")
        print(result['answer'])
        print(format_citation_list(result['citations']))
        print("\n📊 Source Breakdown:")
        for source, count in result['source_breakdown'].items():
            print(f"   {source}: {count} document(s)")

# Create multi-source RAG
retrievers_config = {
    "papers": (papers_store, 0.4),
    "blogs": (blog_store, 0.3),
    "docs": (doc_store, 0.3)
}

multi_rag = MultiSourceRAG(llm, retrievers_config, default_k=2)

# Test it
question = "Explain RAG and how to implement it in practice"
result = multi_rag.query(question)
multi_rag.print_result(result)

❓ Question: Explain RAG and how to implement it in practice

📝 Answer:
Retrieval Augmented Generation (RAG) is a method that combines the capabilities of large language models (LLMs) with external knowledge retrieval to enhance the quality of generated content. By integrating external knowledge retrieval, RAG helps reduce hallucinations and improves the accuracy and relevance of the information provided [1]. This approach is particularly useful for generating more up-to-date and reliable content.

To implement RAG in practice, several key considerations need to be taken into account. When building production RAG systems, it is important to consider various factors such as chunking strategies, embedding models, retrieval algorithms, and prompt engineering. Each of these choices plays a crucial role in determining the overall quality and effectiveness of the RAG system [3].

One practical way to implement RAG is through the use of LangChain's RetrievalQA chain. This chain combines a retr

## Part 6: Reranking for Better Retrieval

**Why Reranking?**
- Initial retrieval (embedding similarity) isn't perfect
- Reranking models better understand query-document relevance
- Significantly improves RAG accuracy
- Popular rerankers: Cohere, Sentence Transformers, cross-encoders

### 6a. Simple Score-Based Reranking

Implement a basic reranker using relevance scores.

In [34]:
from langchain_core.documents import Document
from typing import List, Tuple

class SimpleReranker:
    """Basic reranker using similarity scores and metadata"""
    
    def __init__(self, vectorstore):
        self.vectorstore = vectorstore
    
    def rerank(self, query: str, documents: List[Document], top_k: int = 4) -> List[Document]:
        """
        Rerank documents based on:
        1. Similarity score
        2. Document type preference (papers > blogs > news)
        3. Recency (newer = better)
        """
        # Get similarity scores
        docs_with_scores = self.vectorstore.similarity_search_with_score(query, k=len(documents)*2)
        
        # Create score map
        doc_scores = {}
        for doc, score in docs_with_scores:
            # Lower score = more similar in some embeddings
            doc_scores[doc.page_content] = score
        
        # Rerank with custom scoring
        ranked_docs = []
        for doc in documents:
            # Base score from similarity
            base_score = doc_scores.get(doc.page_content, 1.0)
            
            # Boost based on document type
            type_boost = {
                'paper': 0.1,
                'documentation': 0.05,
                'blog': 0.02,
                'news': 0.0
            }.get(doc.metadata.get('document_type', ''), 0)
            
            # Boost based on recency
            year = doc.metadata.get('year', 2020)
            recency_boost = (year - 2020) * 0.01
            
            # Calculate final score (lower is better)
            final_score = base_score - type_boost - recency_boost
            
            ranked_docs.append((doc, final_score))
        
        # Sort by score and return top_k
        ranked_docs.sort(key=lambda x: x[1])
        return [doc for doc, score in ranked_docs[:top_k]]

# Test reranking
reranker = SimpleReranker(vectorstore)

question = "How do I use vector databases?"

# Get initial results
initial_docs = vectorstore.similarity_search(question, k=6)
print("📋 Initial Retrieval (top 6):")
for i, doc in enumerate(initial_docs, 1):
    print(f"{i}. [{doc.metadata['document_type']}] {doc.metadata['title']}")

print("\n" + "="*70 + "\n")

# Rerank to get top 4
reranked_docs = reranker.rerank(question, initial_docs, top_k=4)
print("⭐ After Reranking (top 4):")
for i, doc in enumerate(reranked_docs, 1):
    print(f"{i}. [{doc.metadata['document_type']}] {doc.metadata['title']}")
    print(f"   Year: {doc.metadata['year']}")

📋 Initial Retrieval (top 6):
1. [paper] Vector Databases for AI Applications
2. [documentation] ChromaDB Documentation - Quick Start
3. [blog] Production RAG Best Practices
4. [paper] RAG: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
5. [documentation] LangChain Documentation - RetrievalQA
6. [blog] Getting Started with LangChain


⭐ After Reranking (top 4):
1. [paper] Vector Databases for AI Applications
   Year: 2024
2. [documentation] ChromaDB Documentation - Quick Start
   Year: 2024
3. [paper] RAG: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
   Year: 2023
4. [blog] Production RAG Best Practices
   Year: 2024


### 6b. Contextual Compression

Use LangChain's built-in compression retriever to filter and compress retrieved content.

In [ ]:
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor

# Create a compressor that extracts only relevant parts
compressor = LLMChainExtractor.from_llm(llm)

# Wrap base retriever with compression
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectorstore.as_retriever(search_kwargs={"k": 4})
)

# Test compression
question = "What are the main benefits of RAG systems?"

print("🔍 Querying with contextual compression...\n")
compressed_docs = compression_retriever.invoke(question)

print(f"Retrieved {len(compressed_docs)} compressed documents:\n")
for i, doc in enumerate(compressed_docs, 1):
    print(f"{i}. {doc.metadata['title']}")
    print(f"   Compressed content: {doc.page_content}")
    print()

🔍 Querying with contextual compression...

Retrieved 2 compressed documents:

1. Production RAG Best Practices
   Compressed content: consider chunking strategies, embedding models, retrieval algorithms, and prompt engineering.

2. RAG: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
   Compressed content: Retrieval Augmented Generation (RAG) combines the power of large language models with external knowledge retrieval. This approach significantly reduces hallucinations and provides more accurate, up-to-date information.



## Part 7: Retrieval Quality Metrics

**Measuring Success:**
- Precision: Are retrieved docs relevant?
- Recall: Did we retrieve all relevant docs?
- MRR (Mean Reciprocal Rank): Position of first relevant doc
- NDCG: Ranking quality

Let's implement basic evaluation metrics.

In [36]:
from typing import List, Set

class RetrievalEvaluator:
    """Evaluate retrieval quality"""
    
    @staticmethod
    def precision_at_k(retrieved_docs: List[str], relevant_docs: Set[str], k: int) -> float:
        """
        Precision@K: What fraction of top-k retrieved docs are relevant?
        """
        top_k = retrieved_docs[:k]
        relevant_in_top_k = len([doc for doc in top_k if doc in relevant_docs])
        return relevant_in_top_k / k if k > 0 else 0.0
    
    @staticmethod
    def recall_at_k(retrieved_docs: List[str], relevant_docs: Set[str], k: int) -> float:
        """
        Recall@K: What fraction of all relevant docs are in top-k?
        """
        top_k = retrieved_docs[:k]
        relevant_in_top_k = len([doc for doc in top_k if doc in relevant_docs])
        return relevant_in_top_k / len(relevant_docs) if len(relevant_docs) > 0 else 0.0
    
    @staticmethod
    def mean_reciprocal_rank(retrieved_docs: List[str], relevant_docs: Set[str]) -> float:
        """
        MRR: 1 / (position of first relevant doc)
        Higher is better. 1.0 means first doc was relevant.
        """
        for i, doc in enumerate(retrieved_docs, 1):
            if doc in relevant_docs:
                return 1.0 / i
        return 0.0

# Example evaluation
question = "What is RAG?"

# Retrieve documents
retrieved = vectorstore.similarity_search(question, k=5)
retrieved_titles = [doc.metadata['title'] for doc in retrieved]

# Define ground truth (which documents SHOULD be retrieved for this query)
relevant_titles = {
    "RAG: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks",
    "Production RAG Best Practices",
    "LangChain Documentation - RetrievalQA"
}

# Evaluate
evaluator = RetrievalEvaluator()

precision = evaluator.precision_at_k(retrieved_titles, relevant_titles, k=5)
recall = evaluator.recall_at_k(retrieved_titles, relevant_titles, k=5)
mrr = evaluator.mean_reciprocal_rank(retrieved_titles, relevant_titles)

print("📊 Retrieval Quality Metrics")
print("="*50)
print(f"Query: {question}")
print(f"\nRetrieved documents:")
for i, title in enumerate(retrieved_titles, 1):
    relevant_marker = "✓" if title in relevant_titles else "✗"
    print(f"  {i}. [{relevant_marker}] {title}")

print(f"\nMetrics:")
print(f"  Precision@5: {precision:.2%} (What fraction of retrieved docs are relevant)")
print(f"  Recall@5: {recall:.2%} (What fraction of relevant docs were retrieved)")
print(f"  MRR: {mrr:.3f} (Reciprocal rank of first relevant doc)")

# F1 Score
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
print(f"  F1 Score: {f1:.2%} (Harmonic mean of precision and recall)")

📊 Retrieval Quality Metrics
Query: What is RAG?

Retrieved documents:
  1. [✓] RAG: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
  2. [✓] Production RAG Best Practices
  3. [✓] LangChain Documentation - RetrievalQA
  4. [✗] Getting Started with LangChain
  5. [✗] Emergent Abilities of Large Language Models

Metrics:
  Precision@5: 60.00% (What fraction of retrieved docs are relevant)
  Recall@5: 100.00% (What fraction of relevant docs were retrieved)
  MRR: 1.000 (Reciprocal rank of first relevant doc)
  F1 Score: 75.00% (Harmonic mean of precision and recall)


## 🎯 Part 8: Exercises & Challenges

Now it's your turn! Try these exercises to reinforce your learning.

### Exercise 1: Build a Multi-Author Knowledge Base

**Task**: Create a document collection with at least 10 documents from 3 different authors on a topic of your choice (e.g., machine learning, web development, finance).

**Requirements**:
1. Include rich metadata (author, year, category, document type)
2. Create a function that retrieves documents by author
3. Build a RAG system that can answer questions and cite specific authors

**Bonus**: Add a feature that shows how many times each author was cited in a response.

In [ ]:
# Your code for Exercise 1 here

### Exercise 2: Implement a Smart Document Router

**Task**: Build a system that automatically routes questions to the most appropriate document collection based on query content.

**Requirements**:
1. Create 3 separate vector stores for different topics (e.g., "python", "databases", "ai")
2. Implement a classifier that determines which store(s) to query
3. Route the query appropriately and return cited results

**Hint**: You can use the LLM itself to classify the query, or use keyword matching.

In [ ]:
# Your code for Exercise 2 here

### Exercise 3: Citation Format Customization

**Task**: Extend the citation system to support different citation formats.

**Requirements**:
1. Implement at least 2 citation formats:
   - APA: Author, A. (Year). Title. Source.
   - MLA: Author. "Title." Source, Year.
2. Allow users to choose their preferred format
3. Update the CitedRAG class to use the selected format

**Bonus**: Add a "footnote" style where citations appear as superscript numbers.

In [ ]:
# Your code for Exercise 3 here

### Exercise 4: Advanced Reranking

**Task**: Implement a more sophisticated reranker that considers query-document semantic similarity.

**Requirements**:
1. Use cross-encoder style scoring (you can simulate with LLM if needed)
2. For each retrieved document, ask the LLM to rate relevance 0-10
3. Rerank based on these scores
4. Compare results with baseline retrieval

**Bonus**: Implement batch processing to make this efficient.

In [ ]:
# Your code for Exercise 4 here

### Exercise 5: Build a Research Assistant

**Task**: Create a comprehensive research assistant that combines all concepts from today.

**Requirements**:
1. Multi-source document collection (papers, blogs, docs)
2. Support for metadata filtering
3. Citation system with multiple formats
4. Reranking for better results
5. Quality metrics tracking

**Features to implement**:
- `search(query, filters)`: Search with optional filters
- `ask(question, sources, citation_format)`: Ask questions with source preferences
- `get_stats()`: Show retrieval quality statistics
- `export_citations(format)`: Export used citations

**Bonus**: Add a conversation mode that maintains context and builds a bibliography of all cited sources.

In [ ]:
# Your code for Exercise 5 here

class ResearchAssistant:
    """
    A comprehensive research assistant with:
    - Multi-source RAG
    - Citation management
    - Reranking
    - Quality metrics
    """
    
    def __init__(self, llm, vectorstores_dict):
        # Initialize your research assistant
        pass
    
    def search(self, query, filters=None):
        """Search across sources with optional filters"""
        pass
    
    def ask(self, question, sources=None, citation_format="numbered"):
        """Ask a question and get a cited answer"""
        pass
    
    def get_stats(self):
        """Get retrieval quality statistics"""
        pass
    
    def export_citations(self, format="txt"):
        """Export all citations used"""
        pass

# Test your research assistant
# assistant = ResearchAssistant(llm, {...})
# result = assistant.ask("What is RAG?")

## 🎓 Summary & Key Takeaways

Congratulations! You've learned advanced RAG techniques. Here's what we covered:

### Key Concepts
1. **Multi-Source RAG**: Query multiple document collections simultaneously
2. **Metadata Filtering**: Use document metadata for targeted retrieval
3. **Citation Systems**: Track and attribute sources properly
4. **Reranking**: Improve retrieval accuracy beyond simple similarity
5. **Quality Metrics**: Measure and evaluate retrieval performance

### Advanced Patterns
- **Ensemble Retrievers**: Combine multiple retrievers with weighted scoring
- **Contextual Compression**: Extract only relevant parts of documents
- **Source Attribution**: Track which documents contributed to answers
- **Custom Rerankers**: Implement domain-specific ranking logic

### Production Considerations
- **Transparency**: Always provide sources for fact-checking
- **Flexibility**: Support different citation formats for different audiences
- **Quality**: Use metrics to continuously improve retrieval
- **Efficiency**: Balance accuracy with performance (reranking is expensive)

---

## 🚀 Next Steps

Tomorrow (Day 5), we'll explore:
- **ReAct Agents**: Combine reasoning and actions
- **Tool Integration**: Give LLMs access to external tools
- **Function Calling**: Structured outputs and API calls
- **Agent Loops**: Iterative problem-solving

### Preparation for Day 5
- Review the agent concepts from LangChain docs
- Think about what tools an AI assistant might need
- Consider problems that require multiple steps to solve

---

## 📚 Additional Resources

**Reading:**
- [LangChain Retrieval Docs](https://python.langchain.com/docs/modules/data_connection/)
- [RAG Paper (Lewis et al.)](https://arxiv.org/abs/2005.11401)
- [Dense Passage Retrieval](https://arxiv.org/abs/2004.04906)

**Advanced Topics:**
- Hybrid search (combining dense and sparse retrieval)
- Query expansion and rewriting
- Multi-hop reasoning over retrieved documents
- Adaptive retrieval (deciding when to retrieve more)

**Tools to Explore:**
- Cohere Rerank API
- Sentence Transformers for reranking
- LlamaIndex for advanced indexing strategies

---

Great work today! 🎉